どうやってリードをつなげるか？

1. 元の配列の類似度を使う
2. consensus unit(s) の類似度を使う
3. ユニットの variant を含めた配列組成の比較を使う
4. `smc.cluster_cons`で置き換えた配列の類似度を使う

---

1. -> やった。`1.1. AVA centromere read overlap with daligner.ipynb` および `1.2. AVA centromere read overlap with daligner [including low cover rate reads].ipynb`。思った以上に上手くいった。もう少し丁寧にやればユニットを考える必要はない気もするが、他の生物種でも同様とは限らないのと、マッピングやオーバーラップの確率を考える上でやはりユニットと variant は重要か。

2. -> composition とあまり変わらない気がする。

3. -> Global にはやった。`3.1. Unit base composition-based overlap.ipynb`。もう少し local な特徴を捉えないと overlap には難しそう。

4. -> `3.2. Split-merge sequence clustering.ipynb`。全ユニットからユニットモデルを構築するのは計算量的に厳しそう。次の一手は、決定論的なヒューリスティクスを使うか、リードを分割するか、先に大まかなクラスタリングを済ませておくか。

---

Overlap filtering をする場合、リードから、通常の overlap における (non-repetitive) k-mer seed のような、局所的かつある程度ユニークな特徴量を設計して、all-vs-all から候補を絞る。

ユニットを単位とした何かにしたいのだが、、

In [56]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython.display import display
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.io as pio
pio.templates.default = 'plotly_white'
import logging
import logzero
logzero.loglevel(logging.DEBUG)

In [2]:
dir_fname = 'work'
import os
os.chdir(dir_fname)

In [34]:
import numpy as np
from copy import copy
import pandas as pd
from BITS.clustering.seq import ClusteringSeq
from BITS.seq.align import EdlibRunner
from BITS.seq.consed import ConsedRunner
from BITS.util.io import save_pickle, load_pickle
from BITS.util.io import load_pickle, save_pickle
import numpy as np
import pandas as pd
from BITS.plot.plotly import make_hist, make_scatter, make_layout, show_plot
from BITS.clustering.seq import ClusteringSeq
import consed
from BITS.seq.align import EdlibRunner
from collections import Counter, defaultdict
from logzero import logger
from dataclasses import dataclass
from typing import List
import random
from copy import copy
from collections import Counter
from dataclasses import dataclass
from logzero import logger
import numpy as np
from vca.types import TRUnit
from BITS.clustering.seq import ClusteringSeq
from BITS.seq.align import EdlibRunner
from BITS.util.io import load_pickle, save_pickle
from BITS.util.proc import NoDaemonPool
from BITS.seq.utils import revcomp

In [18]:
def read_to_forward_units(read):
    """Return all units contained in <read> so that starnd of every unit is same as the forward master unit."""
    return [read.seq[unit.start:unit.end] if unit.id == 0 else revcomp(read.seq[unit.start:unit.end])
            for unit in read.units]

In [25]:
sync_reads = load_pickle("centromere_reads_incl_low_cover_rate_all_sync.pkl")

In [43]:
len(sync_reads)

1905

In [8]:
sync_units = []
for read in sync_reads:
    sync_units += read_to_forward_units(read)

In [9]:
len(sync_units)

46538

In [ ]:
centromere_phreds = load_pickle("centromere_phreds_incl_low_cover_rate_all_sync.pkl")

In [ ]:
sync_quals = []
for read in sync_reads:
    sync_quals += [centromere_phreds[read.name][unit.start:unit.end] if unit.id == 0
                   else np.flip(centromere_phreds[read.name][unit.start:unit.end])
                   for unit in read.units]

-> この数ではクラスタリングも厳しい。

In [4]:
db_prefix = "DMEL_CSS"
db_fname = f"{db_prefix}.db"
las_fname = f"TAN.{db_prefix}.las"
from vca import ReadViewer
v = ReadViewer(db_fname, las_fname)

In [26]:
sync_reads_by_id = {read.id: read for read in sync_reads}

In [29]:
v.show(read=sync_reads_by_id[73])

[I 190919 19:02:33 log:17] Starting distance matrix calculation 
[I 190919 19:02:33 log:19] Finished distance matrix calculation


In [28]:
sync_reads_by_id[762]

TRRead(seq='aaatccttcaaaaagtgatagtaatcgttagcgctggtaataagctgttaaaaaccgttattcttacatctatatgaccattttagccgagtaatgacgaaaattccgattgtaaatatcaacttttaggcaaaatccgtttttccaaatttcggtcatcaaataatcagttttttttttttgccacaactttataaatatttgtctgaatacggaatgtcatacttcgctgagctcgtaattacatttacaattaaaatgtgttcaaaaatggaaattatgtttttttgccatacttttcaaatttttatggggggtcattacaaaaaatacgaaaatttacccaaaaattgatttccctaaatccttcaataagtgatagtaatcgttagcgctggtaataagctgttaaaaaccgttattcttacatctatatgaccattttagccgagtaatgacgaaaattccgtttgtaaatatcaactttttggcaaaatccgtttttccaaatttcggtcatcaaataatcagttttttttttgccacaactttataaatatttgtctgaatacggaatgtcatacctcgctgagctcgtaattacatttaaaattaaaatgtgttcaaaaatggaaattatgttttttgccatactttgcaaatttttatggggggtcattacaaaaaatacgaaatttgacccaaatattgatttccctaaatccttcaaaaagtgatagtaatcgttagcgctggtaataagctgttaaaaaccgttattcttacatctatatgaccattttagccgagtaatgacgaaaattccgattgtaaatatcaacttttaggcaaaatccgttttttccaaatttcggtcatcaaataatcagtttttttttttttgccacaactttataaatatttgtctgaatacggaatgtcatacttcgctgagctcgtaattacatttacaattaaaatgtgttca

In [78]:
!(cat master_unit)

caaaaatgttgatatttacaaacgaaattttcgttataacttggctaaaaatggtcacatagatgtaagaataactgttttgagcagctaattaccagtgctaacgatccctattactttttgaaggatttagggaaattaatttttggatcaattttcgcattttttgtaaggaggggggtcatcaaaatttgcaaaatatggccaaaaaatttaatttccatttttgaacacagtttgattggaaattttattacgagctcagtgaggtatgacattccatattcagacaattattttttatgttgtggcaaaataaatgattatttgatgaccgaaatttggaaaaacagactctg


In [79]:
master_unit = "caaaaatgttgatatttacaaacgaaattttcgttataacttggctaaaaatggtcacatagatgtaagaataactgttttgagcagctaattaccagtgctaacgatccctattactttttgaaggatttagggaaattaatttttggatcaattttcgcattttttgtaaggaggggggtcatcaaaatttgcaaaatatggccaaaaaatttaatttccatttttgaacacagtttgattggaaattttattacgagctcagtgaggtatgacattccatattcagacaattattttttatgttgtggcaaaataaatgattatttgatgaccgaaatttggaaaaacagactctg"